In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.\
config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
.enableHiveSupport().getOrCreate()

In [3]:
s3_bucket = "s3a://dend-jose/"
key = "I94_data_transformed_2016.parquet"
df_immigration =spark.read.load(s3_bucket + key, header=True, inferschema=True, format='parquet')

In [4]:
df_immigration.show(5)

+------------+--------------+-------------+-------------------+---------+----------+-----------+--------------------+------------------+-----------------+------------------+-----------------+----------------------+
|arrival_date|departure_date|arrival_month|origin_airport_code|visa_code|stay_state|origin_city|      origin_country|    avg_population| avg_foreign_born|avg_household_size|   avg_median_age|origin_avg_temperature|
+------------+--------------+-------------+-------------------+---------+----------+-----------+--------------------+------------------+-----------------+------------------+-----------------+----------------------+
|  2016-01-05|    2016-01-10|          1.0|                BOA|      2.0|        TX|       Boma|Congo (Democratic...|258439.02197802198|53106.42490842491|2.8451282051282023|33.37948717948723|                25.672|
|  2016-01-05|          null|          1.0|                BOA|      3.0|        TX|       Boma|Congo (Democratic...|258439.02197802198|5310

In [5]:
df_immigration.createOrReplaceTempView('immigration_table')

### What is the average temperature of the country of origin of the top foreign visitors on April? 

In [6]:
spark.sql('''
SELECT COUNT(*) AS count_country, origin_country, origin_avg_temperature
FROM immigration_table
WHERE origin_country != 'United States' AND
arrival_date >= '2016-04-01' AND
arrival_date <= '2016-04-30'
GROUP BY origin_country, origin_avg_temperature
ORDER by count_country DESC
''').show(5)

+-------------+--------------+----------------------+
|count_country|origin_country|origin_avg_temperature|
+-------------+--------------+----------------------+
|       292243|       Nigeria|                27.995|
|       148637|         Chile|                13.267|
|        80111|       Morocco|    16.665999999999997|
|        24656|        Mexico|                25.925|
|        24058|       Ireland|                 6.815|
+-------------+--------------+----------------------+
only showing top 5 rows



### What is the average foreign borns in the top visited states?

In [7]:
spark.sql('''
SELECT COUNT(*) AS count_state, stay_state, avg_foreign_born
FROM immigration_table
GROUP BY stay_state, avg_foreign_born
ORDER by count_state DESC
''').show(5)

+-----------+----------+------------------+
|count_state|stay_state|  avg_foreign_born|
+-----------+----------+------------------+
|    7779538|        CA|54821.985207100595|
|    6805370|        FL| 35340.38738738739|
|    1624228|        TX| 53106.42490842491|
|    1567010|        NY| 318275.4259259259|
|    1003505|        GU|              null|
+-----------+----------+------------------+
only showing top 5 rows



### Which are the top countries that visit the US during the winter for tourism?

In [9]:
spark.sql('''
SELECT COUNT(*) AS count_country, origin_country
FROM immigration_table
WHERE origin_country != 'United States' AND
arrival_date >= '2016-01-01' AND
arrival_date <= '2016-03-20' AND
visa_code = 2
GROUP BY origin_country
ORDER by count_country DESC
''').show(5)

+-------------+--------------+
|count_country|origin_country|
+-------------+--------------+
|       577276|       Nigeria|
|       315593|       Morocco|
|       236138|         Chile|
|        22152|         Spain|
|        22117|     Venezuela|
+-------------+--------------+
only showing top 5 rows



### Which is the country of origin of most students during the summer?

In [10]:
spark.sql('''
SELECT COUNT(*) AS count_country, origin_country
FROM immigration_table
WHERE origin_country != 'United States' AND
arrival_date >= '2016-06-21' AND
arrival_date <= '2016-09-22' AND
visa_code = 3
GROUP BY origin_country
ORDER by count_country DESC
''').show(5)

+-------------+--------------+
|count_country|origin_country|
+-------------+--------------+
|       101127|       Nigeria|
|        56430|         Chile|
|         4713|       Germany|
|         1066|     Venezuela|
|         1066|         Spain|
+-------------+--------------+
only showing top 5 rows

